In [1]:
import pandas as pd

In [2]:
import deepchem as dc

2023-02-01 15:14:24.999992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 15:14:25.613289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-01 15:14:25.613400: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-01 15:14:28.049574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Cargo dataset

In [3]:
df = pd.read_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [4]:
df

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.00,750.00,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.00,100.00,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.00,50000.00,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.00,300.00,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.00,800.00,6.096910
...,...,...,...,...,...
5818,CHEMBL4859103,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCCNC(=O)c4cc(...,2.57,2.57,8.590067
5819,CHEMBL4863615,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)Nc4cn[n...,2.39,2.39,8.621602
5820,CHEMBL4854913,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)N4CCNC(...,0.41,0.41,9.387216
5821,CHEMBL4848527,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)NCc4cc[...,0.63,0.63,9.200659


# Pongo el dataset en el formato que deepchem necesita
Quizas aca se podía bajar directo en el formato

In [5]:
dataset = dc.data.NumpyDataset(X=df['canonical_smiles'].values, y=df['pIC50'].values, ids=df['canonical_smiles'].values)

# Divido en train y test
Faltaría validación estrictametne

In [6]:
splitter = dc.splits.RandomSplitter()

In [7]:
train, test = splitter.train_test_split(dataset)

# Genero el diccionario de smiles que necesita el modelo para contruir el embedding

In [8]:
smiles_dict, lenght = dc.models.TextCNNModel.build_char_dict(dataset)

In [9]:
smiles_dict

{'#': 1,
 '(': 2,
 ')': 3,
 '+': 4,
 '-': 5,
 '/': 6,
 '1': 7,
 '2': 8,
 '3': 9,
 '4': 10,
 '5': 11,
 '6': 12,
 '7': 13,
 '8': 14,
 '=': 15,
 'C': 16,
 'F': 17,
 'H': 18,
 'I': 19,
 'N': 20,
 'O': 21,
 'P': 22,
 'S': 23,
 '[': 24,
 '\\': 25,
 ']': 26,
 '_': 27,
 'c': 28,
 'Cl': 29,
 'Br': 30,
 'n': 31,
 'o': 32,
 's': 33,
 '@': 34,
 '.': 35,
 'a': 36,
 'B': 37,
 'e': 38,
 'i': 39}

# Instancio el modelo

In [ ]:
import mlflow.keras
mlflow.autolog()
#! mlflow ui

In [10]:
model = dc.models.TextCNNModel(
    1, smiles_dict, lenght, mode='regression', n_embedding=128, kernel_sizes=[3, 4, 5], num_filters=[128, 128, 128], dropout=0.,
    log_frequency=5, batch_size=128, optimizer=dc.models.optimizers.Adam()
)

2023-02-01 15:14:30.729821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-01 15:14:30.729884: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-01 15:14:30.729916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (461db338b5fe): /proc/driver/nvidia/version does not exist
2023-02-01 15:14:30.730426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 248)]        0           []                               
                                                                                                  
 dtnn_embedding (DTNNEmbedding)  (None, 248, 128)    5120        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 246, 128)     49280       ['dtnn_embedding[0][0]']         
                                                                                                  
 conv1d_1 (Conv1D)              (None, 245, 128)     65664       ['dtnn_embedding[0][0]']         
                                                                                              

In [12]:
# Es necesario para ver resultados mientras entrana
vc = dc.models.ValidationCallback(test, 20, dc.metrics.r2_score)

# Entreno

In [13]:
model.fit(train, 100, callbacks=[vc])

Step 20 validation: metric-1=-0.166836


TypeError: 'function' object is not subscriptable

# Evaluó

In [ ]:
model.evaluate(train, dc.metrics.r2_score)

In [ ]:
model.evaluate(test, dc.metrics.r2_score)